# Claim Approval Process


In [7]:
# To ignore any warnings
import warnings
warnings.filterwarnings('ignore')
#To run all the commands in a same shell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from matplotlib import pyplot
import numpy as np
import pandas as pd
from scipy.stats import norm
# For plotting graphs
import matplotlib.pyplot as plt

%matplotlib inline
# For data visualization
import seaborn as sns
#for different algorithms
from sklearn.utils import shuffle
#from factor_analyzer import FactorAnalyzer
#from sklearn import decomposition, preprocessing
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import scale
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier

# Import Dataset

In [10]:
df= pd.read_csv(r'C:/Users/Pavan/CSV_FILES_PYTHON/Test data Set_2020-02-17.csv')


In [11]:
#df.Claimstatus=df.Claimstatus.map({'C':1,'D':0}) 
df.isnull().sum()

ClaimID                0
AGMID                  0
LOB                    0
State             162701
ProgramCode            0
PlanCode               0
VehicleMake         1482
VehicleModel        1671
VehicleYear            0
TermMonth              0
TermMiles              0
ComponentCode          0
ComponentSet           2
ContractStatus         0
AgeAtClaim             0
AgeAtCancel            0
ClaimDate              0
PaidDate            5454
AgeofPayment           0
dtype: int64

In [12]:
df.drop_duplicates(['ClaimID', 'ComponentSet'],keep= 'last', inplace = True)


In [14]:
df.columns

Index(['ClaimID', 'AGMID', 'LOB', 'State', 'ProgramCode', 'PlanCode',
       'VehicleMake', 'VehicleModel', 'VehicleYear', 'TermMonth', 'TermMiles',
       'ComponentCode', 'ComponentSet', 'ContractStatus', 'AgeAtClaim',
       'AgeAtCancel', 'ClaimDate', 'PaidDate', 'AgeofPayment'],
      dtype='object')

In [15]:
nulls = df.isnull().sum()
nulls[nulls > 0]

State           119195
VehicleMake        782
VehicleModel       901
ComponentSet         2
PaidDate          4054
dtype: int64

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 123439 entries, 2 to 169936
Data columns (total 19 columns):
ClaimID           123439 non-null int64
AGMID             123439 non-null int64
LOB               123439 non-null object
State             4244 non-null object
ProgramCode       123439 non-null object
PlanCode          123439 non-null object
VehicleMake       122657 non-null object
VehicleModel      122538 non-null object
VehicleYear       123439 non-null int64
TermMonth         123439 non-null int64
TermMiles         123439 non-null int64
ComponentCode     123439 non-null object
ComponentSet      123437 non-null object
ContractStatus    123439 non-null object
AgeAtClaim        123439 non-null int64
AgeAtCancel       123439 non-null int64
ClaimDate         123439 non-null int64
PaidDate          119385 non-null float64
AgeofPayment      123439 non-null int64
dtypes: float64(1), int64(9), object(9)
memory usage: 18.8+ MB


In [18]:
df.shape

(123439, 19)

In [19]:
from sklearn.utils import shuffle
df = shuffle(df)

In [20]:
df.columns,df.shape

(Index(['ClaimID', 'AGMID', 'LOB', 'State', 'ProgramCode', 'PlanCode',
        'VehicleMake', 'VehicleModel', 'VehicleYear', 'TermMonth', 'TermMiles',
        'ComponentCode', 'ComponentSet', 'ContractStatus', 'AgeAtClaim',
        'AgeAtCancel', 'ClaimDate', 'PaidDate', 'AgeofPayment'],
       dtype='object'), (123439, 19))

# Missing values at individual variable level

In [23]:
#shows the missing values at individual variable level

#df.isnull().sum() --> works same as below code
df.apply(lambda x: sum(x.isnull()), axis=0)

ClaimID                0
AGMID                  0
LOB                    0
State             119195
ProgramCode            0
PlanCode               0
VehicleMake          782
VehicleModel         901
VehicleYear            0
TermMonth              0
TermMiles              0
ComponentCode          0
ComponentSet           2
ContractStatus         0
AgeAtClaim             0
AgeAtCancel            0
ClaimDate              0
PaidDate            4054
AgeofPayment           0
dtype: int64

In [24]:
#shows the total number of missing values
df.isnull().sum().sum()

124934

In [25]:
#variables which we can remove
var_null_pc = df.isnull().sum(axis=0).sort_values( ascending=False)/float(len(df) )
var_null_pc[ var_null_pc > 0.75 ]

State    0.965619
dtype: float64

In [1]:
df.drop( var_null_pc[ var_null_pc > 0.75 ].index, axis = 1,  ) 
df.dropna( axis = 0, thresh = 30, )

NameError: name 'df' is not defined

# COLUMNS REMOVED

In [27]:
#The Variables which can be removed are -,
vars_to_be_removed = [ 'ClaimDate', 'PaidDate', 'AGMID',  'State','AgeofPayment','AgeAtCancel',
 'AgeAtClaim','ComponentCode'
 ]

df.drop(vars_to_be_removed , axis = 1, inplace=True )#inplace=True

In [28]:
#shows the variable names present in the dataframe
list(df.columns.values)

['ClaimID',
 'LOB',
 'ProgramCode',
 'PlanCode',
 'VehicleMake',
 'VehicleModel',
 'VehicleYear',
 'TermMonth',
 'TermMiles',
 'ComponentSet',
 'ContractStatus']

In [29]:
df.columns

Index(['ClaimID', 'LOB', 'ProgramCode', 'PlanCode', 'VehicleMake',
       'VehicleModel', 'VehicleYear', 'TermMonth', 'TermMiles', 'ComponentSet',
       'ContractStatus'],
      dtype='object')

In [30]:
# find categorical variables
categorical = [var for var in df.columns if df[var].dtype=='O']
print('There are {} categorical variables'.format(len(categorical)))

There are 7 categorical variables


In [31]:

# find numerical variables
numerical = [var for var in df.columns if df[var].dtype!='O']
print('There are {} numerical variables'.format(len(numerical)))

There are 4 numerical variables


In [32]:
df[categorical].columns

Index(['LOB', 'ProgramCode', 'PlanCode', 'VehicleMake', 'VehicleModel',
       'ComponentSet', 'ContractStatus'],
      dtype='object')

In [33]:
df[numerical].columns

Index(['ClaimID', 'VehicleYear', 'TermMonth', 'TermMiles'], dtype='object')

In [34]:
df.shape

(123439, 11)

# One hot encoding and dummy encoding on Categorical

In [35]:
#Changing the datatype of the variable in a dataframe
df['ProgramCode'] = df['ProgramCode'].astype('category')
#df['ProductCode'] = df['ProductCode'].astype('category')
df['PlanCode'] = df['PlanCode'].astype('category')

#df['State'] = df['State'].astype('category')
df['ContractStatus'] = df['ContractStatus'].astype('category')
df['ComponentSet'] = df['ComponentSet'].astype('category')
df['LOB'] = df['LOB'].astype('category')
df['VehicleMake'] = df['VehicleMake'].astype('category')
df['VehicleModel'] = df['VehicleModel'].astype('category')
#df['FuelType'] = df['FuelType'].astype('category') 
#df['ComponentCode'] = df['ComponentCode'].astype('category')

In [36]:
# Convert all non-numeric values to number
cat=['LOB',  'ProgramCode', 'PlanCode', 'VehicleMake',
       'VehicleModel', 'ContractStatus']

In [37]:
from sklearn import preprocessing
le =preprocessing.LabelEncoder()
for var in cat:
    le = preprocessing.LabelEncoder()
    df[var]=le.fit_transform(df[var].astype('str'))
df.dtypes

ClaimID              int64
LOB                  int32
ProgramCode          int32
PlanCode             int32
VehicleMake          int32
VehicleModel         int32
VehicleYear          int64
TermMonth            int64
TermMiles            int64
ComponentSet      category
ContractStatus       int32
dtype: object

In [38]:
# explicitly require this experimental feature
from sklearn.experimental import enable_iterative_imputer  # noqa
# now you can import normally from sklearn.impute
from sklearn.impute import IterativeImputer
from sklearn.ensemble import ExtraTreesRegressor
#from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import OrdinalEncoder

In [39]:
df.isnull().sum()

ClaimID           0
LOB               0
ProgramCode       0
PlanCode          0
VehicleMake       0
VehicleModel      0
VehicleYear       0
TermMonth         0
TermMiles         0
ComponentSet      2
ContractStatus    0
dtype: int64

In [40]:

# outlies in discrete variables
for var in [  'VehicleYear', 'TermMonth', 'TermMiles',    
       ]:
    print(df[var].value_counts() / np.float(len(df)))
    print()

2012    0.098275
2011    0.097311
2013    0.097052
2014    0.089145
2008    0.080072
2007    0.072287
2015    0.068576
2010    0.065304
2006    0.058701
2009    0.055930
2005    0.042458
2016    0.037946
2004    0.029189
2017    0.022456
2003    0.018495
2018    0.011325
2002    0.011196
2001    0.007858
0       0.006351
2000    0.005930
1999    0.005590
2019    0.004618
1998    0.003710
1997    0.003419
1996    0.002706
1995    0.001928
1994    0.000875
1993    0.000575
1992    0.000275
1990    0.000227
2020    0.000154
1991    0.000065
Name: VehicleYear, dtype: float64

60     0.345928
48     0.191892
36     0.113821
72     0.099166
24     0.073883
120    0.054432
84     0.050851
38     0.031311
1      0.011536
12     0.011334
999    0.004739
96     0.003953
75     0.002560
6      0.001215
39     0.000834
168    0.000826
3      0.000794
66     0.000348
42     0.000203
63     0.000138
78     0.000081
30     0.000057
54     0.000041
18     0.000024
57     0.000008
69     0.000008
65   

In [41]:
for var in categorical:
    print(var, ' contains ', len(df[var].unique()), ' labels')

LOB  contains  3  labels
ProgramCode  contains  214  labels
PlanCode  contains  502  labels
VehicleMake  contains  61  labels
VehicleModel  contains  3846  labels
ComponentSet  contains  56  labels
ContractStatus  contains  10  labels


In [2]:
df2=pd.concat([df, pd.get_dummies(df.ComponentSet)], 1).groupby(['ClaimID']).sum().reset_index()
df2

NameError: name 'pd' is not defined

In [43]:
df2.drop([ 'LOB', 'ProgramCode', 'PlanCode', 'VehicleMake',
       'VehicleModel', 'VehicleYear', 'TermMonth', 'TermMiles',
       'ContractStatus'], axis=1, inplace=True) 

In [3]:
df1=df.groupby(['ClaimID']).last()
df1

NameError: name 'df' is not defined

In [47]:
df1.drop([ 'ComponentSet'], axis=1, inplace=True) 


In [48]:
df1.columns

Index(['LOB', 'ProgramCode', 'PlanCode', 'VehicleMake', 'VehicleModel',
       'VehicleYear', 'TermMonth', 'TermMiles', 'ContractStatus'],
      dtype='object')

In [4]:
df2.columns

NameError: name 'df2' is not defined

In [5]:
df3=df1.merge(df2, left_on='ClaimID', right_on='ClaimID')
df3.columns

NameError: name 'df1' is not defined

In [51]:
df3.to_csv('C:/Users/Pavan/CSV_FILES_PYTHON/OUTPUT/2Test_merge2.0.csv',index=False)

In [44]:
#The Variables which can be removed are -, removed Catogerical  ComponentSet_OTHER column to avoid dummy variable trap
#vars_to_be_removed = [ 'ComponentSet']

#df.drop(vars_to_be_removed , axis = 1, inplace=True )#inplace=True

# END Test DATA CleanUP